# Combine dat files from unattended ApRES surveys on Thwaites glacier into netcdfs for archiving
### J. Kingslake July 2nd 2025
The code below collates the dat filed collected in unattneded ApRES surveys on Thwaites Glacier between 2022 and 2024 by the GHOST team, part of the International Thwaites Glacier Collaboration. The data were collected by Elizabeth Case, Columbia University, then Utrecht University, Andrew Hoffman, Columbia University, then Rice University, and Ole Zeisling, Alfred Wegner Institute.

In [ ]:
import xapres as xa
import xarray as xr
import os

Define where the dat files are and where we will write the netcdfs.

In [ ]:
sites = ["ApRES_LTG", "ApRES_Lake1", "ApRES_Lake2", "ApRES_Takahe1_204", "ApRES_Takahe2_203"]
source = "/Users/jkingslake/Documents/data/thwaites_apres/original/continuous/"
archive_location ='/Users/jkingslake/Documents/data/thwaites_apres/archiving/unattended'

Define a function for changing the format of the attributes in the xarray to work nicely when written to netcdf. 

In [ ]:
def remove_constants_attr(ds):
    for c in ds.attrs['constants']:
        ds.attrs[c] = ds.attrs['constants'][c]
    del ds.attrs['constants']
    return ds


Loop through the loacations and
1. find all the dat files, subset them, and sort them by filename, so that they are in time order. 
2. Split the list of dat files into blocks with a size that results in reasonably sized netcdfs. 
3. Define netcdf paths and make directoies for them
4. Delete previously created netcdfs 
5. loop through the blocks of files and load all the dat files into an xarray using the package xapres
6. add the stacked chirps
7. tidy up the units and attributes to work well with netcdfs
8. write the netcdfs
(the last three steps are performed for each block of files) 

In [ ]:

tg_1 = xa.load.from_dats()
for site in sites:
    # 1. find all the dat files, subset them, and sort them by filename, so that they are in time order. 
    dat_directory = os.path.join(source, site, "dat_files")
    filepaths = tg_1.list_files(directory=dat_directory)
    print(f"there are {len(filepaths)} files from {site}")
    one_dat = tg_1.load_all(directory = dat_directory, file_numbers_to_process=[7], computeProfiles = False, disable_progress_bar=True)
    print(f"total size of dataset without profiles computed = {one_dat.nbytes/1e9 * len(filepaths):.2f} GB in memory")

    if site == 'ApRES_LTG': 
        start_dat_file_number = 4  # the first 4 use the same attenuator settings, but they are not regularly spaces in time with the other data
    elif site == 'ApRES_Lake1':
        start_dat_file_number = 3  # the first 3 use different attenuator settings. THis is the first one which is regularly spaced in time with the other data
    elif site == 'ApRES_Lake2':
        start_dat_file_number = 4  # This is the first one which is regularly spaced in time with the other data
    elif site == 'ApRES_Takahe1_204':
        start_dat_file_number = 0
    elif site == 'ApRES_Takahe2_203':
        start_dat_file_number = 3 # something is up with the first 3 files some seem to have no data and one is the wrong size. 
    else:
        raise ValueError(f"Unknown site {site}")

    dat_size = os.path.getsize(filepaths[start_dat_file_number+5])
    print(f"dat file size = {dat_size/1e6:.2f} MB on disk")

    f = sorted(filepaths)[start_dat_file_number:]

    # 2. Split the list of dat files into blocks with a size that results in reasonably sized netcdfs. 
    N = int(17*104e6/dat_size)  # number of files to process in each block
    file_lists = [f[i:i + N] for i in range(0, len(f), N)]
    print(f'created {len(file_lists)} file lists from {len(f)} files')
    print(f'list sizes: {[len(x) for x in file_lists]}')

    # 3. Define netcdf paths and make directoies for them
    nc_locations = [f'{archive_location}/{site}/netcdf/part_{i}.nc' for i in range(len(file_lists))]
    os.makedirs(f"{archive_location}/{site}/netcdf", exist_ok=True) 
    
    # 4. Delete previously created netcdfs 
    for nc in nc_locations:
        if os.path.exists(nc):
            print(f"deleting existing file {nc}")
            os.remove(nc)

    # 5. loop through the blocks of files and load all the dat files into an xarray using the package xapres
    for i, file_list in enumerate(file_lists):
        print(f"Processing {len(file_list)} files")
        ds = tg_1.load_all(directory = dat_directory, 
                        computeProfiles = False,
                        file_names_to_process=file_list,
                        disable_progress_bar=False)
        
        # 6. add the stacked chirps
        ds['chirp_stacked'] = ds.chirp.mean(dim='chirp_num', keep_attrs=True)
        
        # 7. tidy up the units and attributes to work well with netcdfs
        ds.chirp_stacked.attrs['long_name'] = 'stacked de-ramped chirp'
        ds["chirp_time"].attrs["units"] = "unscaled seconds"
        ds = remove_constants_attr(ds)
        
        # 8. write the netcdfs
        ds.to_netcdf(nc_locations[i])


## Reload the data
To combine the multiple netcdfs into one xarray, use xarray's `open_mfdataset`.

In [ ]:
ds_r = xr.open_mfdataset(f"{archive_location}/{sites[3]}/netcdf/*")
ds_r